In [1]:
import numpy as np
import skimage.io as io
import skimage.segmentation
import skimage.morphology
import os

Prior to running this notebook, perform Cell Detection in QuPath using DAPI channel as a nuclear marker. Then export the rendered image of the nuclear segmentation map (with all nuclear channels diabled) as tiff files and move them to a folder (in_dir). This notebook is used to convert the nuclear segmentation map that QuPath renders into a binary nuclear mask that can be used for evaluation and comparison with other nuclear segmentation platforms.

An alternative way to arriving at the binary nuclear mask is to use a groovy script which can automate this process in QuPath. We chose to use the former method.

In [2]:
def QuPath_postprocessor(in_dir, save_dir):
    '''
    Converts nuclear segmentation map rendered in QuPath GUI to a binary nuclear mask to be used for evaluation

    Parameters
    in_dir: str
        the path to the directory containing the nuclear segmentation map images rendered in QuPath
    save_dir: str
        the path to the directory where the binary masks will be saved. Each tiff image in the input directory will
        have a corresponding binary mask in the save directory
    '''
    
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the QuPath-rendered image as an array
            platform_img = io.imread(os.path.join(in_dir, file))
            # creating a nuclear mask where all the nucleus body regions are white/foreground. The nucleaus border
            # and non-nuclei regions are black/background.
            nuclei_mask = np.uint8(np.zeros((platform_img.shape[0], platform_img.shape[1])))
            r, g, b = platform_img[:,:,0], platform_img[:,:,1], platform_img[:,:,2]
            mask = ((r == 255) & (g == 255) & (b == 255))
            nuclei_mask[mask] = 255
            # The nucleus border needs to also be white/foreground in the final binary mask. Additionally, nuclei
            # that are touching need to have one pixel eroded from either nucleus.
            labels = skimage.morphology.label(nuclei_mask)
            labels = skimage.segmentation.relabel_sequential(labels)[0]
            labels = skimage.segmentation.expand_labels(labels, distance=2)
            boundary_bool = skimage.segmentation.find_boundaries(labels, connectivity=2, mode='outer', background=0)
            labels[boundary_bool] = 0
            nuclei_mask_final = np.uint8(np.zeros((platform_img.shape[0], platform_img.shape[1])))
            nuclei_mask_final[labels != 0] = 255
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

In [3]:
# Replace the strings for in_dir and save_dir to paths on the local machine
in_dir = '/Users/abishek/Desktop/MRL/Platforms/Code/nuclear_seg/QuPath_rendered_masks'
save_dir = '/Users/abishek/Desktop/MRL/Platforms/Code/nuclear_seg/QuPath_binary_masks'
# To create binary masks from QuPath rendered nuclear segmentation map images
QuPath_postprocessor(in_dir, save_dir)